In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

## Parte 1: Extract & Load

Nesta parte, iremos ler os CSVs e carregar tudo em uma tabela "bruta" (stg_samu) dentro de um banco SQLite temporário.

In [2]:
# Aqui farei a conexão com o banco de dados

conn = sqlite3.connect('samu_dw.db')

In [3]:
# Definirei aqui, nomes padrão para garantir que todos tenham as mesmas colunas
colunas_padrao = [
    '_id', 'data', 'hora_minuto', 'municipio', 'bairro', 'endereco',
    'origem_chamado', 'tipo', 'subtipo', 'sexo', 'idade',
    'motivo_finalizacao', 'motivo_desfecho'
]

In [4]:
# Farei o carregamento das bases que serão utilizadas
# Garantir que todas as bases tenham as mesmas colunas para evitar problemas no concat/to_sql

df_23 = pd.read_csv('../data/samu_2023.csv', header=None, names=colunas_padrao)
df_24 = pd.read_csv('../data/samu_2024.csv', header=None, names=colunas_padrao)
df_25 = pd.read_csv('../data/samu_2025.csv', header=None, names=colunas_padrao)

C:\Users\sofia\AppData\Local\Temp\ipykernel_3132\3206089797.py:4: DtypeWarning: Columns (0,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_23 = pd.read_csv('../data/samu_2023.csv', header=None, names=colunas_padrao)
C:\Users\sofia\AppData\Local\Temp\ipykernel_3132\3206089797.py:5: DtypeWarning: Columns (0,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_24 = pd.read_csv('../data/samu_2024.csv', header=None, names=colunas_padrao)
C:\Users\sofia\AppData\Local\Temp\ipykernel_3132\3206089797.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_25 = pd.read_csv('../data/samu_2025.csv', header=None, names=colunas_padrao)


In [5]:
# Adicionarei colunas de origem

df_23['ano_origem'] = 2023
df_24['ano_origem'] = 2024
df_25['ano_origem'] = 2025

In [6]:
# Utilizarei pd.concat() para juntar todos os dataframes

df_total = pd.concat([df_23, df_24, df_25])

df_total.head()

,_id,data,hora_minuto,municipio,bairro,endereco,origem_chamado,tipo,subtipo,sexo,idade,motivo_finalizacao,motivo_desfecho,ano_origem
0,_id,data,hora_minuto,municipio,bairro,endereco,origem_chamado,tipo,subtipo,sexo,idade,motivo_finalizacao,motivo_desfecho,2023
1,1,2023-01-01T00:00:00,00:03:18,JABOATAO DOS GUARARAPES,JARDIM MURIBECA,R FLOR DE LARANJEIRA,RESIDENCIAL,GERAIS/OUTROS,OUTROS,FEMININO,16,REGULAÇÃO POR TELEFONE,SEM DESFECHO,2023
2,2,2023-01-01T00:00:00,00:11:40,TIMBAUBA,CENTRO,AV AGAMENOM MAGALHAES CASA,RESIDENCIAL,PSIQUIATRICA,AGITACAO,MASCULINO,44,NaN,OCORRÊNCIA CONCLUÍDA COM ÊXITO,2023
3,3,2023-01-01T00:00:00,00:13:21,RECIFE,CORDEIRO,R ODETE MONTEIRO,RESIDENCIAL,GERAIS/OUTROS,OUTROS,FEMININO,82,NaN,ACOMPANHANTE RECUSA REMOÇÃO,2023
4,4,2023-01-01T00:00:00,00:15:11,RECIFE,PINA,AV BOA VIAGEM SN,VIA PÚBLICA,CAUSAS EXTERNAS,OUTROS,FEMININO,19,NaN,OCORRÊNCIA CONCLUÍDA COM ÊXITO,2023


#### PRÉ-TRANSFORMAÇÃO 
    
Agora, realizaremos o cálculo e o preenchimento da idade pela mediana
- **Justificativa**: O cálculo da mediana é mais fácil e rápido no Pandas, realizar esse processo em SQL seria muito trabalhoso e deixaria o processo mais lento.

In [ ]:
# PRÉ-LIMPEZA CRÍTICA: Remove linhas de cabeçalho duplicadas que contaminaram os dados.
    # Justificativa: O string 'idade' impede o cálculo da mediana. Filtramos todas as linhas que contêm o nome da coluna.
df_total = df_total[df_total['idade'].astype(str).str.lower() != 'idade']

try:
    df_total['idade'] = pd.to_numeric(df_total['idade'], errors='coerce')
    
    # Agora calculamos a mediana (somente sobre os números)
    mediana_idade = df_total['idade'].median()
    df_total['idade'].fillna(mediana_idade, inplace=True)
    df_total.to_sql('stg_samu_raw', conn, if_exists='replace', index=False)
    
    print(f"✅ Sucesso! {len(df_total)} registros carregados na tabela 'stg_samu_raw'.")

except Exception as e:
    print(f"❌ Erro na extração: {e}")

C:\Users\sofia\AppData\Local\Temp\ipykernel_3132\2899212134.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_total['idade'].fillna(mediana_idade, inplace=True)


✅ Sucesso! 539519 registros carregados na tabela 'stg_samu_raw'.


In [8]:
# Jogarei aqui os dados brutos no banco de dados
# if_exists='replace' garante que recriamos a tabela se rodarmos de novo

df_total.to_sql('stg_samu', conn, if_exists='replace', index=False)

539519

## Parte 2: Transform

Agora que os dados estão no banco, iremos limpar e criar o Esquema Estrela (Star Schema) usando SQL.

In [9]:
# Criarei uma função para executar SQL

def run_sql(query):
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        conn.commit()
    except Exception as e:
        print(f"Erro: {e}")

### 1. Dimensão Tempo

In [10]:
# Extrairei o dia, mês, ano e dia da semana da data bruta

dimensao_tempo = """
CREATE TABLE IF NOT EXISTS dim_tempo AS
SELECT DISTINCT
    data as id_tempo,
    strftime('%Y', data) as ano,
    strftime('%m', data) as mes,
    strftime('%d', data) as dia,
    strftime('%w', data) as dia_semana
FROM stg_samu
WHERE data IS NOT NULL
"""

In [11]:
run_sql(dimensao_tempo)

### 2. Dimensão Local

In [12]:
# Normalizarei os nomes (tudo maiúsculo) para evitar duplicatas como "Recife" e "RECIFE"

# Criarei a tabela
run_sql("""
CREATE TABLE IF NOT EXISTS dim_local (
    id_local INTEGER PRIMARY KEY AUTOINCREMENT,
    municipio TEXT,
    bairro TEXT
)
""")

# Insere os dados
dimensao_local = """
INSERT INTO dim_local (municipio, bairro)
SELECT DISTINCT
    UPPER(municipio),
    UPPER(bairro)
FROM stg_samu
WHERE municipio IS NOT NULL
"""

In [13]:
run_sql(dimensao_local)

### 3. Dimensão Motivo

In [14]:
# Aqui iremos limpar os números do campo 'motivo_desfecho' (ex: '1. SUCESSO' vira 'SUCESSO')
# e trataremos dados nulos com 'NAO INFORMADO'

# Criarei a tabela
run_sql("""
CREATE TABLE IF NOT EXISTS dim_motivo (
    id_motivo INTEGER PRIMARY KEY AUTOINCREMENT,
    tipo TEXT,
    subtipo TEXT,
    motivo_desfecho_limpo TEXT
)
""")

# Insere os dados
dimensao_motivo = """
INSERT INTO dim_motivo (tipo, subtipo, motivo_desfecho_limpo)
SELECT DISTINCT
    UPPER(tipo),
    UPPER(COALESCE(subtipo, 'NAO INFORMADO')),
    CASE
        WHEN motivo_desfecho LIKE '%1. %' THEN SUBSTR(motivo_desfecho, 4)
        WHEN motivo_desfecho LIKE '%2. %' THEN SUBSTR(motivo_desfecho, 4)
        WHEN motivo_desfecho LIKE '%3. %' THEN SUBSTR(motivo_desfecho, 4)
        ELSE UPPER(motivo_desfecho)
    END as motivo_limpo
FROM stg_samu
"""

In [15]:
run_sql(dimensao_motivo)

### 4. Tabela Fato (Fato Atendimentos)

In [16]:
# Unirei tudo conectando os IDs das dimensões e calculando as métricas

fato = """
CREATE TABLE IF NOT EXISTS fato_atendimentos AS
SELECT 
    s._id as id_atendimento,
    s.data as id_tempo,
    l.id_local,
    m.id_motivo,
    s.idade,
    s.sexo,
    s.hora_minuto,
    s.ano_origem
FROM stg_samu s
LEFT JOIN dim_local l 
    ON UPPER(s.municipio) = l.municipio AND UPPER(s.bairro) = l.bairro
LEFT JOIN dim_motivo m
    ON UPPER(s.tipo) = m.tipo 
    AND UPPER(COALESCE(s.subtipo, 'NAO INFORMADO')) = m.subtipo
    AND (
        (s.motivo_desfecho LIKE '%1. %' AND m.motivo_desfecho_limpo = SUBSTR(s.motivo_desfecho, 4)) OR
        (m.motivo_desfecho_limpo = UPPER(s.motivo_desfecho))
    );
"""

In [17]:
run_sql(fato)

Agora, farei um Sanity Check para avaliar se a tabela tem dados.

In [18]:
check_query = "SELECT COUNT(*) as total_linhas FROM fato_atendimentos;"
cursor = conn.cursor()
cursor.execute(check_query)
print(f"Total de registros na Tabela Fato: {cursor.fetchone()[0]}")

# Garantir que as chaves (IDs) não estão nulas
df_check = pd.read_sql("SELECT * FROM fato_atendimentos LIMIT 5;", conn)
display(df_check)

Total de registros na Tabela Fato: 539518


,id_atendimento,ano_origem,id_tempo,id_local,id_motivo,idade,sexo,hora_minuto
0,1,2023,2023-01-01T00:00:00,1,1,16,FEMININO,00:03:18
1,2,2023,2023-01-01T00:00:00,2,2,44,MASCULINO,00:11:40
2,3,2023,2023-01-01T00:00:00,3,3,82,FEMININO,00:13:21
3,4,2023,2023-01-01T00:00:00,4,4,19,FEMININO,00:15:11
4,5,2023,2023-01-01T00:00:00,5,1,79,MASCULINO,00:16:04


Aqui, vemos que o Sanity funcionou e revelou que a tabela possui "sujeira". Provavelmente o arquivo de 2023 tinha alguma linha duplicada de cabeçalho ou o processo de leitura interpretou o cabeçalho como uma linha de dados. Isso é "lixo" que entraria na análise e sujaria os gráficos.

Agora, vamos remover essa linha "intrusa" e depois gerar os gráficos.

In [19]:
# Correção

cursor = conn.cursor()
cursor.execute("DELETE FROM fato_atendimentos WHERE id_atendimento = '_id'")
conn.commit()

In [20]:
df_check_pos = pd.read_sql("SELECT * FROM fato_atendimentos LIMIT 5;", conn)
display(df_check_pos)

,id_atendimento,ano_origem,id_tempo,id_local,id_motivo,idade,sexo,hora_minuto
0,1,2023,2023-01-01T00:00:00,1,1,16,FEMININO,00:03:18
1,2,2023,2023-01-01T00:00:00,2,2,44,MASCULINO,00:11:40
2,3,2023,2023-01-01T00:00:00,3,3,82,FEMININO,00:13:21
3,4,2023,2023-01-01T00:00:00,4,4,19,FEMININO,00:15:11
4,5,2023,2023-01-01T00:00:00,5,1,79,MASCULINO,00:16:04


## Parte 3: Visualização

O próximo passo é cumprir o requisito de "Apresentação de Três Análises e Insights".

Agora que o Data Warehouse está criado e populado, é preciso interrogar os dados para gerar valor. A seguir, vou gerar as 3 análises e os gráficos.

In [21]:
# A princípio, vamos configurar o ambiente para visualização

sns.set_theme(style="whitegrid")
plt.figure(figsize=(20, 10))

<Figure size 2000x1000 with 0 Axes>

<Figure size 2000x1000 with 0 Axes>